# Welcome to the SLRfield package

This package is an archive of scientific routines for data processing related to SLR(Satellite Laser Ranging).   
Currently, operations on SLR data include:
    1. Automatically download the CPF(Consolidated Prediction Format) ephemeris file
    2. Parse the CPF ephemeris file
    3. Predict the azimuth, altitude, distance of the target, and the time of flight for laser pulse etc. given the coordinates of the station    

## How to Install

SLRfield can be installed with `pip install slrfield`.

## How to Upgrade

SLRfield can be updated to the latest version with `pip install slrfield --upgrade`.

## How to use

### Download the latest CPF ephemeris files at the current moment

In this package, for prediction data centers, only **CDDIS**(Crustal Dynamics Data Information System) and **EDC**(EUROLAS Data Center) are available. If the prediction data center is not provided, then it is set to **CDDIS** by default.

#### Download all available targets from CDDIS by default

Note: the data storage directory(such as CPF/CDDIS/2020-10-02) will be automatically created or emptied(if exist) by default ahead of requesting and downloading CPF files .

In [ ]:
from slrfield import cpf_download
cpf_files_all = cpf_download()

Note: the data storage directory will be created/kept ahead of requesting CPF files by setting `append = True`.

In [ ]:
cpf_files_append = cpf_download(['beaconc','lageos1'],append=True)

#### Download all available targets from EDC

In [ ]:
cpf_files_all = cpf_download(source = 'EDC') 

#### Download a set of specified targets from CDDIS by default

In [ ]:
from slrfield import cpf_download
sat_lists = ['ajisai','lageos1','hy2a','etalon2','jason3','beaconc','larets']
cpf_files_cddic = cpf_download(sat_lists) 

#### Download a set of specified targets from EDC

In [ ]:
sat_lists = ['ajisai','lageos1','hy2a','etalon2','jason3']
cpf_files_edc = cpf_download(sat_lists,source = 'EDC',append=True) 

### Download the latest CPF ephemeris files before a specific date and time

From ***CDDIS*** by default

In [ ]:
sat_name = 'lageos1'
date = '2007-06-01 11:30:00'
cpf_file_cddis = cpf_download(sat_name,date)

From ***EDC***

In [ ]:
sat_lists = ['starlette','lageos1']
date = '2017-01-01 11:30:00'
cpf_files_edc = cpf_download(sat_lists,date,'EDC',append=True)

### Parse the CPF ephemeris files and read the data

Information from the parsed CPF ephemeris files includes the following contents:
 - Format
 - Format Version
 - Ephemeris Source
 - Date and time of ephemeris production
 - Ephemeris Sequence number
 - Target name
 - COSPAR ID
 - SIC
 - NORAD ID
 - Starting date and time of ephemeris
 - Ending date and time of ephemeris
 - Time between table entries (UTC seconds)
 - Target type
 - Reference frame
 - Rotational angle type
 - Center of mass correction
 - Direction type
 - Modified Julian Date
 - Second of Day 
 - Leap Second
 - Time moment in UTC 
 - Target positions in meters

#### Parse a single CPF ephemeris file

In [ ]:
from slrfield import read_cpfs
cpf_data_cddis = read_cpfs(cpf_file_cddis)
print(cpf_data_cddis.info)

#### Parse a set of CPF ephemeris files

In [ ]:
cpf_data_edc = read_cpfs(cpf_files_edc)
print(cpf_data_edc.target_name)
print(cpf_data_edc.ephemeris_source)
print(cpf_data_edc.norad_id)
print(cpf_data_edc.cospar_id)
print(cpf_data_edc.time_ephemeris) # Date and time of ephemeris production
print(cpf_data_edc.version) # Format Version

### Make predictions

The azimuth, altitude, distance of a target w.r.t. a given station, and the time of flight for laser pulse etc. can be easily predicted by calling a method `pred`. The output prediction files named with target names are stored in directory pred by default. 

- There are two modes for the prediction. If the mode is set to ***geometric***, then the transmitting direction of the laser will coincide with the receiving direction at a certain moment. In this case, the output prediction file will not contain the difference between the receiving direction and the transmitting direction. If the mode is set to ***apparent***, then the transmitting direction of the laser is inconsistent with the receiving direction at a certain moment. In this case, the output prediction file will contain the difference between the receiving direction and the transmitting direction. The default mode is set to ***apparent***.
- The 10-point(degree 9) Lagrange polynomial interpolation method is used to interpolate the CPF ephemeris.
- Effects of leap second have been considered in the prediction generation.

Coordinates of station can either be ***geocentric***(x, y, z) in meters or ***geodetic***(lon, lat, height) in degrees and meters. The default coordinates type is set to ***geodetic***.

#### For geodetic(lon, lat, height) station coordinates by default

In [ ]:
t_start = '2007-06-01 17:06:40'
t_end = '2007-06-02 09:06:40'
t_increment = 0.5 # second

station = [46.877230,7.465222,951.33] # geodetic(lon, lat, height) coordinates in degrees and meters by default
cpf_data_cddis.pred(station,t_start,t_end,t_increment)

####  For geocentric(x, y, z) station coordinates

In [ ]:
t_start = '2017-01-01 22:06:40'
t_end = '2017-01-02 09:06:40'
t_increment = 2 # second

station = [4331283.557, 567549.902,4633140.353] # geocentric(x, y, z) coordinates of station in meters
cpf_data_edc.pred(station,t_start,t_end,t_increment,coord_type = 'geocentric',mode='geometric')

#### Calculate the visible passes for target transit

In [ ]:
from slrfield import target_query,tle_download,visible_pass
# Targets filtering by a set of parameters
targets_df = target_query(Payload=False,Decayed=False,RCSAvg=[5,8],MeanAlt=[300,600])

# Download TLE/3LE data
noradids = list(targets_df['NORADID'])
tle_download(noradids)

# Calculate visible passes for target transit
t_start = '2020-06-10 12:00:00' # starting date and time 
t_end = '2020-06-17 00:00:00' # ending date and time
timezone = 8 # timezone
# station position in lat[deg], lon[deg], and height[m] above the WGS84 ellipsoid
site = ['25.03 N','102.80 E','1987.05'] 
visible_pass(t_start,t_end,site,timezone)